# stacathome for site-specific downloads of ECOSTRESS

To run this script, 

1) install stacathome as descibed in the readme. 
    1.1) activate environment:  source .venv/bin/activate
    1.2) install ipykernel: uv pip install ipykernel
    1.3) install a kernel you can use in this notebook: python -m ipykernel install --user --name stacathome --display-name "stacathome" 

2) create an earthaccess account: https://www.earthdata.nasa.gov/data/tools/earthaccess
3) import earthaccess and then run earthaccess.login(persist=True) to write the credentials to your home folder so you'll stay logged in. 

In [ ]:
import earthaccess
earthaccess.login(persist=True)

We select a region of interest, in this example Johannesburg (SA; UTM point: 603053,7099667 ), but for downloading for example over an active Fluxnet site, check this page:
https://www.icos-etc.eu/home/sites-list

We need to specify a buffer around the site that will be targeted for download. Specify size of the box for your buffer and resolution to indicate size.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from datetime import datetime
import shapely
from test.processors.planetary.sentinel_2_l2a.test_processor import create_test_geobox
from stacathome.requests import _get_provider_and_processor

import stacathome

geobox = create_test_geobox(shapely.Point(603053,7099667), resolution=100, size_box=5000, crs='EPSG:32735')
roi = geobox.footprint('EPSG:4326', buffer=0, npoints=4)


Because here we are interested in the coverage of the diurnal cycle, we access and download images over all 5 years available in ECOSTRESS. We only use version 002 (ECO_002 in the filename), the latest one. processor.filter_items() will apply a a filter that only keeps the image from the latest processing iteration (indicated by e.g. the 0712_01_ in the filename). 

Select a startdate and an end date, here the dry season in this specific area, e.g. June and September. 
You request a provider and a processor

In [ ]:
for i in range(2019, 2024):
    for month in [6, 9]:
        startdate = datetime(i, month, 1)
        enddate = datetime(i, month+1, 1)

        provider_name = 'earthaccess'
        collection = 'ECO_L2T_LSTE'
        kwargs = {'version': '002'}


        provider, processor = _get_provider_and_processor(provider_name, collection, None, False)

        items = provider.request_items(
            collection=collection,
            starttime=startdate,
            endtime=enddate,
            roi=roi,
            **kwargs,
        )
        if not items:
            raise ValueError('No items matched the search query')

        items = processor.filter_items(provider, roi, items)

        provider.load_granule(
            items,
            out_dir='/Net/Groups/BGI/work_5/scratch/mwessel/ecostress', # specify path on which to store sdata
            threads=4)

Check how many items the processor found in the requested time period over the region of interest:

In [ ]:
len(items)
items
provider.load_granule(items)

Make use of the geobox to explore the cutout in your region of interest.

In [ ]:
roi.explore()

Explore this data with stacathome as you loaded it:

In [ ]:
collection, dataset = stacathome.load(
    provider_name='earthaccess',
    collection='ECO_L2T_LSTE',
    version ='002',
    roi=roi,
    starttime=startdate,
    endtime=enddate,
    variables=['LST'],
)
dataset

# Examples for Sentinel 2

In [ ]:
from stacathome.processors import BaseProcessor
collection, dataset = stacathome.load(
    provider_name='cdse',
    collection='sentinel-2-l2a',
    roi=roi,
    starttime=startdate,
    endtime=enddate,
    variables=['B08_10m', 'B8A_20m', 'B01_60m'],
    # processor=BaseProcessor(),
)
dataset

In [ ]:
collection

In [ ]:
collection_pc, dataset_pc = stacathome.load(
    provider_name='planetary_computer',
    collection='sentinel-2-l2a',
    roi=roi,
    starttime=startdate,
    endtime=enddate,
    variables=['B08', 'B8A', 'B01'],
)
dataset_pc

In [ ]:
collection_pc

In [ ]:
np.unique((dataset.B01_60m.isel(time=2) - 1000 ).values == dataset_pc.B01.isel(time=0).values)

In [ ]:
dataset.B01_60m.isel(time=3).plot()

In [ ]:
# startdate = datetime(2024, 7, 10)
# enddate = datetime(2024, 7, 30)

# collection, dataset = stacathome.load(
#     provider_name='earthaccess',
#     collection='ECO_L2T_LSTE',
#     version ='002',
#     roi=roi,
#     starttime=startdate,
#     endtime=enddate,
#     variables=['LST'],
# )
# dataset

import shapely
from stacathome.providers import get_provider
from stacathome.processors.ecostress import ECO_L2T_LSTEProcessor
from stacathome.stac import enclosing_geoboxes_per_grid
from test.processors.planetary.sentinel_2_l2a.test_processor import create_test_geobox

variables = ['LST']
provider = get_provider('earthaccess')

geobox = create_test_geobox(shapely.Point(603053,7099667), resolution=100, size_box=5000, crs='EPSG:32735')
roi = geobox.footprint('EPSG:4326', buffer=0, npoints=4)

items = provider.request_items(
    collection='ECO_L2T_LSTE',
    version ='002',
    starttime='2025-07-10',
    endtime='2025-07-15',
    roi=roi,
)

processor = ECO_L2T_LSTEProcessor()

u_items = processor.filter_items(provider, roi, items)

# #-< enter solution 2 get all granules - the next line will download all selected variables in the filtered u_items
u_items = provider.load_granule(u_items, variables)

# # now the items assets are updated (just for the selected variables!) so creating a zarr from them local:

gb = enclosing_geoboxes_per_grid(u_items[0], roi)
data = processor.load_items_geoboxed(provider, gb[0].enclosing_box, u_items, variables = variables)
# data.to_zarr('/Net/Groups/BGI/work_5/scratch/mzehner/eco_MLESM/ecostress_test.zarr', mode='w')